In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import re  #regular expression

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q16-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q16-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 84 columns):
 #   Column                                                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                                                --------------  -----  
 0   Response ID                                                                                                                                                           396 non-null    int64  
 1   Live streaming:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?                                   3 non-null      float64
 2   Collaboration with Influencer:BOSCH:GM - And which of these types of content have you seen recently for these 

In [4]:
Germany.head(2) #initial dataset

,Response ID,Live streaming:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:BOSCH:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:MIELE:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:MIELE:GM - And which of these types of content have you seen recently for these brands when looking at social media?,...,Tik Tok content:HAIER:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:HAIER:GM - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):HAIER:GM - And which of these types of content have you seen recently for these brands when looking at social media?",Live streaming:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:ROWENTA:GM - And which of these types of content have you seen recently for these brands when looking at social media?
0,78230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_GM['SCM_Platform_cleaned'] = unpivoted_GM['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_GM = unpivoted_GM[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_GM['SCM_Platform_cleaned'] = unpivoted_GM['SCM_Platform_cleaned'].str.replace(
    ":GM - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

       Response ID                                       SCM_Platform  \
32863        94276  None of the above:ROWENTA:GM - And which of th...   
32864        94287  None of the above:ROWENTA:GM - And which of th...   
32865        94291  None of the above:ROWENTA:GM - And which of th...   
32866        95452  None of the above:ROWENTA:GM - And which of th...   
32867        95582  None of the above:ROWENTA:GM - And which of th...   

            SCM_Platform_cleaned  Y/N  
32863  None of the above:ROWENTA  NaN  
32864  None of the above:ROWENTA  NaN  
32865  None of the above:ROWENTA  NaN  
32866  None of the above:ROWENTA  NaN  
32867  None of the above:ROWENTA  NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_GM['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
count = unpivoted_GM['Y/N'].value_counts()
print(count)

Y/N
5.0     135
3.0     133
4.0     105
6.0      83
2.0      75
1.0      54
99.0     49
Name: count, dtype: int64


In [8]:
# Drop the original 'SCM_Platform' column
unpivoted_GM = unpivoted_GM.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_GM = unpivoted_GM[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_GM['Y/N'] = np.where(unpivoted_GM['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_GM['Y/N'] == 99, 99,  
                        np.where(unpivoted_GM['Y/N'].isna(), 0, unpivoted_GM['Y/N'])))

In [9]:
unpivoted_GM.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78230,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
1,78263,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
2,78268,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
3,78278,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH
4,78299,Live streaming:BOSCH:GM - And which of these t...,Live streaming,0.0,BOSCH


In [10]:
count = unpivoted_GM['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32234
1.0       585
99.0       49
Name: count, dtype: int64


Brand
BOSCH        2772
MIELE        2772
SIEMENS      2772
AEG          2772
SAMSUNG      2772
BAUKNECHT    2772
PHILIPS      2772
LG           2772
DYSON        2772
NEFF         2772
ROWENTA      2772
HAIER        2376
Name: count, dtype: int64


Total number of rows: 32868


### Sweden

In [11]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    int64  
 1   Live streaming:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?                               7 non-null      float64
 2   Collaboration with Influencer:ELECTROLUX:SW - And which of these types of content have you seen recently f

In [12]:
Sweden.head(2) #initial dataset

,Response ID,Live streaming:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:ELECTROLUX:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:BOSCH:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:SW - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:AEG:SW - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):AEG:SW - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:AEG:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:ROBOROCK:SW - And which of these types of content have you seen recently for these brands when looking at social media?
0,78236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_SW['SCM_Platform_cleaned'] = unpivoted_SW['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_SW = unpivoted_SW[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_SW['SCM_Platform_cleaned'] = unpivoted_SW['SCM_Platform_cleaned'].str.replace(
    ":SW - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

       Response ID                                       SCM_Platform  \
33595        95995  None of the above:ROBOROCK:SW - And which of t...   
33596        96005  None of the above:ROBOROCK:SW - And which of t...   
33597        96010  None of the above:ROBOROCK:SW - And which of t...   
33598        96062  None of the above:ROBOROCK:SW - And which of t...   
33599        96068  None of the above:ROBOROCK:SW - And which of t...   

             SCM_Platform_cleaned  Y/N  
33595  None of the above:ROBOROCK  NaN  
33596  None of the above:ROBOROCK  NaN  
33597  None of the above:ROBOROCK  NaN  
33598  None of the above:ROBOROCK  NaN  
33599  None of the above:ROBOROCK  NaN  


In [14]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_SW['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [15]:
count = unpivoted_SW['Y/N'].value_counts()
print(count)

Y/N
3.0     195
5.0     159
6.0     129
2.0      80
99.0     63
4.0      62
1.0      60
Name: count, dtype: int64


In [16]:
# Drop the original 'SCM_Platform' column
unpivoted_SW = unpivoted_SW.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_SW = unpivoted_SW[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SW['Y/N'] = np.where(unpivoted_SW['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_SW['Y/N'] == 99, 99,  
                        np.where(unpivoted_SW['Y/N'].isna(), 0, unpivoted_SW['Y/N'])))

In [17]:
unpivoted_SW.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78236,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
1,78238,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
2,78241,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
3,78246,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX
4,78248,Live streaming:ELECTROLUX:SW - And which of th...,Live streaming,0.0,ELECTROLUX


In [18]:
count = unpivoted_SW['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32852
1.0       685
99.0       63
Name: count, dtype: int64


Brand
ELECTROLUX    2800
BOSCH         2800
SAMSUNG       2800
MIELE         2800
PHILIPS       2800
SIEMENS       2800
CYLINDA       2800
WHIRLPOOL     2800
LG            2800
DYSON         2800
AEG           2800
ROBOROCK      2800
Name: count, dtype: int64


Total number of rows: 33600


### Italy

In [19]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 84 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            398 non-null    int64  
 1   Live streaming:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?                                  12 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:IT - And which of these types of content have you seen recently for 

In [20]:
Italy.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:BOSCH:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:IT - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:HAIER:IT - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):HAIER:IT - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:HAIER:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SIEMENS:IT - And which of these types of content have you seen recently for these brands when looking at social media?
0,78233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_IT['SCM_Platform_cleaned'] = unpivoted_IT['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_IT = unpivoted_IT[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_IT['SCM_Platform_cleaned'] = unpivoted_IT['SCM_Platform_cleaned'].str.replace(
    ":IT - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

       Response ID                                       SCM_Platform  \
33029        93279  None of the above:SIEMENS:IT - And which of th...   
33030        93284  None of the above:SIEMENS:IT - And which of th...   
33031        93290  None of the above:SIEMENS:IT - And which of th...   
33032        93341  None of the above:SIEMENS:IT - And which of th...   
33033        93357  None of the above:SIEMENS:IT - And which of th...   

            SCM_Platform_cleaned  Y/N  
33029  None of the above:SIEMENS  NaN  
33030  None of the above:SIEMENS  NaN  
33031  None of the above:SIEMENS  NaN  
33032  None of the above:SIEMENS  NaN  
33033  None of the above:SIEMENS  NaN  


In [22]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_IT['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [23]:
count = unpivoted_IT['Y/N'].value_counts()
print(count)

Y/N
3.0     256
5.0     173
6.0     136
4.0     124
2.0      89
1.0      83
99.0     53
Name: count, dtype: int64


In [24]:
# Drop the original 'SCM_Platform' column
unpivoted_IT = unpivoted_IT.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_IT = unpivoted_IT[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_IT['Y/N'] = np.where(unpivoted_IT['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_IT['Y/N'] == 99, 99,  
                        np.where(unpivoted_IT['Y/N'].isna(), 0, unpivoted_IT['Y/N'])))

In [25]:
unpivoted_IT.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78233,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
1,78237,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
2,78242,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
3,78300,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG
4,78315,Live streaming:SAMSUNG:IT - And which of these...,Live streaming,0.0,SAMSUNG


In [26]:
count = unpivoted_IT['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32120
1.0       861
99.0       53
Name: count, dtype: int64


Brand
SAMSUNG       2786
BOSCH         2786
WHIRLPOOL     2786
LG            2786
DYSON         2786
MIELE         2786
ELECTROLUX    2786
HOTPOINT      2786
BEKO          2786
HAIER         2786
SIEMENS       2786
AEG           2388
Name: count, dtype: int64


Total number of rows: 33034


### UK

In [27]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 84 columns):
 #   Column                                                                                                                                                               Non-Null Count  Dtype  
---  ------                                                                                                                                                               --------------  -----  
 0   Response ID                                                                                                                                                          396 non-null    int64  
 1   Live streaming:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?                                17 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:UK - And which of these types of content have you seen recently for these br

In [28]:
UK.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:LG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:LG:UK - And which of these types of content have you seen recently for these brands when looking at social media?,...,Tik Tok content:SIEMENS:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SIEMENS:UK - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SIEMENS:UK - And which of these types of content have you seen recently for these brands when looking at social media?",Live streaming:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:NEFF:UK - And which of these types of content have you seen recently for these brands when looking at social media?
0,78251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_UK['SCM_Platform_cleaned'] = unpivoted_UK['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_UK = unpivoted_UK[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_UK['SCM_Platform_cleaned'] = unpivoted_UK['SCM_Platform_cleaned'].str.replace(
    ":UK - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

       Response ID                                       SCM_Platform  \
32863        94064  None of the above:NEFF:UK - And which of these...   
32864        95433  None of the above:NEFF:UK - And which of these...   
32865        95591  None of the above:NEFF:UK - And which of these...   
32866        95639  None of the above:NEFF:UK - And which of these...   
32867        96098  None of the above:NEFF:UK - And which of these...   

         SCM_Platform_cleaned  Y/N  
32863  None of the above:NEFF  NaN  
32864  None of the above:NEFF  NaN  
32865  None of the above:NEFF  NaN  
32866  None of the above:NEFF  NaN  
32867  None of the above:NEFF  NaN  


In [30]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_UK['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [31]:
count = unpivoted_UK['Y/N'].value_counts()
print(count)

Y/N
3.0     201
5.0     183
6.0     171
4.0     133
2.0     103
1.0      94
99.0     35
Name: count, dtype: int64


In [32]:
# Drop the original 'SCM_Platform' column
unpivoted_UK = unpivoted_UK.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_UK = unpivoted_UK[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_UK['Y/N'] = np.where(unpivoted_UK['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_UK['Y/N'] == 99, 99,  
                        np.where(unpivoted_UK['Y/N'].isna(), 0, unpivoted_UK['Y/N'])))

In [33]:
unpivoted_UK.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78251,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
1,78258,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
2,78259,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,1.0,SAMSUNG
3,78261,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG
4,78262,Live streaming:SAMSUNG:UK - And which of these...,Live streaming,0.0,SAMSUNG


In [34]:
count = unpivoted_UK['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     31948
1.0       885
99.0       35
Name: count, dtype: int64


Brand
SAMSUNG     2772
LG          2772
HOTPOINT    2772
BOSCH       2772
DYSON       2772
BEKO        2772
SHARK       2772
ZANUSSI     2772
MIELE       2772
AEG         2772
NEFF        2772
SIEMENS     2376
Name: count, dtype: int64


Total number of rows: 32868


### Poland

In [35]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 85 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    int64  
 1   Live streaming:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?                                  13 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:PO - And which of these types of content have you seen recently for 

In [36]:
Poland.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:BOSCH:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:BOSCH:PO - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:MIELE:PO - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):MIELE:PO - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:MIELE:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:AEG:PO - And which of these types of content have you seen recently for these brands when looking at social media?
0,78295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_PO['SCM_Platform_cleaned'] = unpivoted_PO['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_PO = unpivoted_PO[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_PO['SCM_Platform_cleaned'] = unpivoted_PO['SCM_Platform_cleaned'].str.replace(
    ":PO - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

       Response ID                                       SCM_Platform  \
33595        96000  None of the above:AEG:PO - And which of these ...   
33596        96002  None of the above:AEG:PO - And which of these ...   
33597        96003  None of the above:AEG:PO - And which of these ...   
33598        96016  None of the above:AEG:PO - And which of these ...   
33599        96017  None of the above:AEG:PO - And which of these ...   

        SCM_Platform_cleaned  Y/N  
33595  None of the above:AEG  NaN  
33596  None of the above:AEG  NaN  
33597  None of the above:AEG  NaN  
33598  None of the above:AEG  NaN  
33599  None of the above:AEG  NaN  


In [38]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_PO['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [39]:
count = unpivoted_PO['Y/N'].value_counts()
print(count)

Y/N
3.0     311
6.0     196
5.0     156
4.0     154
1.0     120
2.0      67
99.0     61
Name: count, dtype: int64


In [40]:
# Drop the original 'SCM_Platform' column
unpivoted_PO = unpivoted_PO.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_PO = unpivoted_PO[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_PO['Y/N'] = np.where(unpivoted_PO['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_PO['Y/N'] == 99, 99,  
                        np.where(unpivoted_PO['Y/N'].isna(), 0, unpivoted_PO['Y/N'])))

In [41]:
unpivoted_PO.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78295,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
1,78297,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
2,78309,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
3,78317,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG
4,78344,Live streaming:SAMSUNG:PO - And which of these...,Live streaming,0.0,SAMSUNG


In [42]:
count = unpivoted_PO['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32535
1.0      1004
99.0       61
Name: count, dtype: int64


Brand
SAMSUNG       2800
BOSCH         2800
ELECTROLUX    2800
BEKO          2800
AMICA         2800
PHILIPS       2800
WHIRLPOOL     2800
LG            2800
DYSON         2800
HAIER         2800
MIELE         2800
AEG           2800
Name: count, dtype: int64


Total number of rows: 33600


### Thailand

In [43]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 78 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            399 non-null    int64  
 1   Live streaming:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?                                       58 non-null     float64
 2   Collaboration with Influencer:LG:TH - And which of these types of content have you seen recently for these

In [44]:
Thailand.head(2) #initial dataset

,Response ID,Live streaming:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:LG:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:MITSUBISHI:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:MITSUBISHI:TH - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:BOSCH:TH - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):BOSCH:TH - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:BOSCH:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:DYSON:TH - And which of these types of content have you seen recently for these brands when looking at social media?
0,78252,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78269,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_TH['SCM_Platform_cleaned'] = unpivoted_TH['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_TH = unpivoted_TH[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_TH['SCM_Platform_cleaned'] = unpivoted_TH['SCM_Platform_cleaned'].str.replace(
    ":TH - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

       Response ID                                       SCM_Platform  \
30718        95265  None of the above:DYSON:TH - And which of thes...   
30719        95350  None of the above:DYSON:TH - And which of thes...   
30720        95488  None of the above:DYSON:TH - And which of thes...   
30721        95492  None of the above:DYSON:TH - And which of thes...   
30722        95815  None of the above:DYSON:TH - And which of thes...   

          SCM_Platform_cleaned  Y/N  
30718  None of the above:DYSON  NaN  
30719  None of the above:DYSON  NaN  
30720  None of the above:DYSON  NaN  
30721  None of the above:DYSON  NaN  
30722  None of the above:DYSON  NaN  


In [46]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_TH['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [47]:
count = unpivoted_TH['Y/N'].value_counts()
print(count)

Y/N
3.0     1127
4.0      856
6.0      787
5.0      643
2.0      627
1.0      560
99.0      41
Name: count, dtype: int64


In [48]:
# Drop the original 'SCM_Platform' column
unpivoted_TH = unpivoted_TH.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_TH = unpivoted_TH[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_TH['Y/N'] = np.where(unpivoted_TH['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_TH['Y/N'] == 99, 99,  
                        np.where(unpivoted_TH['Y/N'].isna(), 0, unpivoted_TH['Y/N'])))

In [49]:
unpivoted_TH.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78252,Live streaming:LG:TH - And which of these type...,Live streaming,0.0,LG
1,78269,Live streaming:LG:TH - And which of these type...,Live streaming,1.0,LG
2,78313,Live streaming:LG:TH - And which of these type...,Live streaming,0.0,LG
3,78345,Live streaming:LG:TH - And which of these type...,Live streaming,0.0,LG
4,78363,Live streaming:LG:TH - And which of these type...,Live streaming,0.0,LG


In [50]:
count = unpivoted_TH['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     26082
1.0      4600
99.0       41
Name: count, dtype: int64


Brand
LG            2793
MITSUBISHI    2793
ELECTROLUX    2793
PANASONIC     2793
SHARP         2793
PHILIPS       2793
HITACHI       2793
HAIER         2793
TEFAL         2793
BOSCH         2793
DYSON         2793
Name: count, dtype: int64


Total number of rows: 30723


### Vietnam

In [51]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 81 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            400 non-null    int64  
 1   Live streaming:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?                                  65 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:VN - And which of these types of content have you seen recently for 

In [52]:
Vietnam.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:PANASONIC:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:PANASONIC:VN - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:BOSCH:VN - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):BOSCH:VN - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:BOSCH:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:MIDEA:VN - And which of these types of content have you seen recently for these brands when looking at social media?
0,78271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78275,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_VN['SCM_Platform_cleaned'] = unpivoted_VN['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_VN = unpivoted_VN[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_VN['SCM_Platform_cleaned'] = unpivoted_VN['SCM_Platform_cleaned'].str.replace(
    ":VN - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

       Response ID                                       SCM_Platform  \
31995        95842  None of the above:MIDEA:VN - And which of thes...   
31996        95844  None of the above:MIDEA:VN - And which of thes...   
31997        95845  None of the above:MIDEA:VN - And which of thes...   
31998        95848  None of the above:MIDEA:VN - And which of thes...   
31999        95852  None of the above:MIDEA:VN - And which of thes...   

          SCM_Platform_cleaned  Y/N  
31995  None of the above:MIDEA  NaN  
31996  None of the above:MIDEA  NaN  
31997  None of the above:MIDEA  NaN  
31998  None of the above:MIDEA  NaN  
31999  None of the above:MIDEA  NaN  


In [54]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_VN['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [55]:
count = unpivoted_VN['Y/N'].value_counts()
print(count)

Y/N
3.0     963
4.0     768
6.0     682
2.0     452
5.0     412
1.0     375
99.0     14
Name: count, dtype: int64


In [56]:
# Drop the original 'SCM_Platform' column
unpivoted_VN = unpivoted_VN.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_VN = unpivoted_VN[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_VN['Y/N'] = np.where(unpivoted_VN['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_VN['Y/N'] == 99, 99,  
                        np.where(unpivoted_VN['Y/N'].isna(), 0, unpivoted_VN['Y/N'])))

In [57]:
unpivoted_VN.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78271,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
1,78275,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
2,78287,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
3,78291,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,0.0,SAMSUNG
4,78358,Live streaming:SAMSUNG:VN - And which of these...,Live streaming,1.0,SAMSUNG


In [58]:
count = unpivoted_VN['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     28334
1.0      3652
99.0       14
Name: count, dtype: int64


Brand
SAMSUNG       2800
PANASONIC     2800
LG            2800
SHARP         2800
MIDEA         2800
PHILIPS       2800
BOSCH         2800
AQUA          2800
TOSHIBA       2400
ELECTROLUX    2400
HITACHI       2400
XIAOMI        2400
Name: count, dtype: int64


Total number of rows: 32000


### Australia

In [59]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 85 columns):
 #   Column                                                                                                                                                                    Non-Null Count  Dtype  
---  ------                                                                                                                                                                    --------------  -----  
 0   Response ID                                                                                                                                                               397 non-null    int64  
 1   Live streaming:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?                                     16 non-null     float64
 2   Collaboration with Influencer:SAMSUNG:AU - And which of these types of content have you seen r

In [60]:
Australia.head(2) #initial dataset

,Response ID,Live streaming:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SAMSUNG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:LG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:LG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,...,Instagram content:HAIER:AU - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):HAIER:AU - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:HAIER:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:AEG:AU - And which of these types of content have you seen recently for these brands when looking at social media?
0,78293,1.0,2.0,NaN,4.0,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_AU['SCM_Platform_cleaned'] = unpivoted_AU['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_AU = unpivoted_AU[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_AU['SCM_Platform_cleaned'] = unpivoted_AU['SCM_Platform_cleaned'].str.replace(
    ":AU - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

       Response ID                                       SCM_Platform  \
33343        95482  None of the above:AEG:AU - And which of these ...   
33344        95486  None of the above:AEG:AU - And which of these ...   
33345        95487  None of the above:AEG:AU - And which of these ...   
33346        95489  None of the above:AEG:AU - And which of these ...   
33347        95490  None of the above:AEG:AU - And which of these ...   

        SCM_Platform_cleaned  Y/N  
33343  None of the above:AEG  NaN  
33344  None of the above:AEG  NaN  
33345  None of the above:AEG  NaN  
33346  None of the above:AEG  NaN  
33347  None of the above:AEG  NaN  


In [62]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_AU['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [63]:
count = unpivoted_AU['Y/N'].value_counts()
print(count)

Y/N
3.0     223
4.0     137
6.0     135
5.0     130
1.0      87
2.0      83
99.0     64
Name: count, dtype: int64


In [64]:
# Drop the original 'SCM_Platform' column
unpivoted_AU = unpivoted_AU.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_AU = unpivoted_AU[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_AU['Y/N'] = np.where(unpivoted_AU['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_AU['Y/N'] == 99, 99,  
                        np.where(unpivoted_AU['Y/N'].isna(), 0, unpivoted_AU['Y/N'])))

In [65]:
unpivoted_AU.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78293,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,1.0,SAMSUNG
1,78361,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
2,78394,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
3,78406,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG
4,78429,Live streaming:SAMSUNG:AU - And which of these...,Live streaming,0.0,SAMSUNG


In [66]:
count = unpivoted_AU['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32489
1.0       795
99.0       64
Name: count, dtype: int64


Brand
SAMSUNG         2779
LG              2779
DYSON           2779
WESTINGHOUSE    2779
BOSCH           2779
HISENSE         2779
MIELE           2779
SMEG            2779
ELECTROLUX      2779
HAIER           2779
AEG             2779
Name: count, dtype: int64


Total number of rows: 33348


In [67]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in Brand:", blank_count)

Count of blank values in Brand: 2779


In [68]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [69]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          2779
LG               2779
DYSON            2779
WESTINGHOUSE     2779
FISHER&PAYKEL    2779
BOSCH            2779
HISENSE          2779
MIELE            2779
SMEG             2779
ELECTROLUX       2779
HAIER            2779
AEG              2779
Name: count, dtype: int64


Total number of rows: 33348


### Switzerland

In [70]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 84 columns):
 #   Column                                                                                                                                                                 Non-Null Count  Dtype  
---  ------                                                                                                                                                                 --------------  -----  
 0   Response ID                                                                                                                                                            398 non-null    int64  
 1   Live streaming:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?                                    9 non-null      float64
 2   Collaboration with Influencer:V ZUG:SZ - And which of these types of content have you seen recently for th

In [71]:
Switzerland.head(2) #initial dataset

,Response ID,Live streaming:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:V ZUG:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Live streaming:ELECTROLUX:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:ELECTROLUX:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,...,Tik Tok content:LIEBHERR:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:LIEBHERR:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):LIEBHERR:SZ - And which of these types of content have you seen recently for these brands when looking at social media?",Live streaming:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Collaboration with Influencer:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Facebook content:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Tik Tok content:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,Instagram content:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?,"Forums (user groups, Facebook groups, etc.):SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?",None of the above:SCHULTHESS:SZ - And which of these types of content have you seen recently for these brands when looking at social media?
0,78375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.0
1,78387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='SCM_Platform', value_name='Y/N')

# Create a duplicate of the 'SCM_Platform' column named 'SCM_Platform_cleaned'
unpivoted_SZ['SCM_Platform_cleaned'] = unpivoted_SZ['SCM_Platform']

# Rearrange columns to place 'SCM_Platform_cleaned' next to 'Source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N']]

# Remove the unwanted suffix from the 'SCM_Platform_cleaned' column only
unpivoted_SZ['SCM_Platform_cleaned'] = unpivoted_SZ['SCM_Platform_cleaned'].str.replace(
    ":SZ - And which of these types of content have you seen recently for these brands when looking at social media?", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

       Response ID                                       SCM_Platform  \
33029        95956  None of the above:SCHULTHESS:SZ - And which of...   
33030        95963  None of the above:SCHULTHESS:SZ - And which of...   
33031        95971  None of the above:SCHULTHESS:SZ - And which of...   
33032        95973  None of the above:SCHULTHESS:SZ - And which of...   
33033        96100  None of the above:SCHULTHESS:SZ - And which of...   

               SCM_Platform_cleaned   Y/N  
33029  None of the above:SCHULTHESS   NaN  
33030  None of the above:SCHULTHESS   NaN  
33031  None of the above:SCHULTHESS   NaN  
33032  None of the above:SCHULTHESS   NaN  
33033  None of the above:SCHULTHESS  99.0  


In [73]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'SCM_Platform_cleaned'
for entry in unpivoted_SZ['SCM_Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [74]:
count = unpivoted_SZ['Y/N'].value_counts()
print(count)

Y/N
5.0     185
3.0     175
4.0     136
6.0     131
2.0     107
1.0      92
99.0     43
Name: count, dtype: int64


In [75]:
# Drop the original 'SCM_Platform' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['SCM_Platform_cleaned'])

# Rename 'Ad_Type' to 'SCM_Platform'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'SCM_Platform_cleaned'})

# Rearrange columns to place 'SCM_Platform' next to 'Platform'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'SCM_Platform', 'SCM_Platform_cleaned', 'Y/N', 'Brand']]

# Replace values in the 'Y/N' column
unpivoted_SZ['Y/N'] = np.where(unpivoted_SZ['Y/N'].isin([1, 2, 3, 4, 5, 6]), 1, 
                        np.where(unpivoted_SZ['Y/N'] == 99, 99,  
                        np.where(unpivoted_SZ['Y/N'].isna(), 0, unpivoted_SZ['Y/N'])))

In [76]:
unpivoted_SZ.head()

,Response ID,SCM_Platform,SCM_Platform_cleaned,Y/N,Brand
0,78375,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
1,78387,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
2,78581,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
3,78735,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None
4,78849,Live streaming:V ZUG:SZ - And which of these t...,Live streaming,0.0,None


In [77]:
count = unpivoted_SZ['Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Y/N
0.0     32165
1.0       826
99.0       43
Name: count, dtype: int64


Brand
ELECTROLUX    2786
MIELE         2786
DYSON         2786
BOSCH         2786
SAMSUNG       2786
SIEMENS       2786
PHILIPS       2786
BAUKNECHT     2786
AEG           2786
SCHULTHESS    2786
LIEBHERR      2388
Name: count, dtype: int64


Total number of rows: 33034


In [78]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 2786


In [79]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [80]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         2786
ELECTROLUX    2786
MIELE         2786
DYSON         2786
BOSCH         2786
SAMSUNG       2786
SIEMENS       2786
PHILIPS       2786
BAUKNECHT     2786
AEG           2786
SCHULTHESS    2786
LIEBHERR      2388
Name: count, dtype: int64


Total number of rows: 33034


### Concat and save them in another csv file

In [81]:
# Concatenate the melted DataFrames
combined_q16 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q16.dropna(subset=['Response ID'], inplace=True)

combined_q16 = combined_q16.sort_values(by='Response ID', ascending=True)

# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q16.xlsx', float_format='%.0f', index=False)
# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q16.xlsx', float_format='%.0f', index=False)
# combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q16.xlsx', float_format='%.0f', index=False)
combined_q16.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q16.xlsx', float_format='%.0f', index=False)

print(combined_q16.head(2))

      Response ID                                       SCM_Platform  \
0           78230  Live streaming:BOSCH:GM - And which of these t...   
8712        78230  Collaboration with Influencer:AEG:GM - And whi...   

               SCM_Platform_cleaned  Y/N  Brand  
0                    Live streaming  0.0  BOSCH  
8712  Collaboration with Influencer  0.0    AEG  


In [82]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q16['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [83]:
combined_q16.info()

<class 'pandas.core.frame.DataFrame'>
Index: 295075 entries, 0 to 196858
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Response ID           295075 non-null  int64  
 1   SCM_Platform          295075 non-null  object 
 2   SCM_Platform_cleaned  295075 non-null  object 
 3   Y/N                   295075 non-null  float64
 4   Brand                 295075 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 13.5+ MB


In [84]:
count = combined_q16['Brand'].value_counts()
print(count)

Brand
BOSCH            25088
LG               22302
SAMSUNG          22295
DYSON            22288
MIELE            19495
ELECTROLUX       19144
AEG              19097
PHILIPS          16751
HAIER            13534
SIEMENS          13520
WHIRLPOOL         8386
BEKO              8358
SHARP             5593
PANASONIC         5593
BAUKNECHT         5558
HOTPOINT          5558
NEFF              5544
HITACHI           5193
MIDEA             2800
ROBOROCK          2800
CYLINDA           2800
AMICA             2800
AQUA              2800
TEFAL             2793
MITSUBISHI        2793
SCHULTHESS        2786
V ZUG             2786
HISENSE           2779
WESTINGHOUSE      2779
SMEG              2779
FISHER&PAYKEL     2779
ROWENTA           2772
ZANUSSI           2772
SHARK             2772
TOSHIBA           2400
XIAOMI            2400
LIEBHERR          2388
Name: count, dtype: int64
